In [47]:
import string
print 'walk'.capitalize() # Walk is good
print 'AWOL'.capitalize() # not good because Awol isn't a word
print '-ness'.capitalize()
print 'February'.capitalize()
al = string.lowercase[0:26]
print type(al)
# for x in al: print x
print 'Tuesday'[0] in al

Walk
Awol
-ness
February
<type 'str'>
False


In [49]:
# output mwl39195hw.txt  (having duplicates)
# and explore 39195 hw entries
# 2015-12-01

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs
import string

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest

fhw = codecs.open('E:\\1Now\\taglist\\mwl39195hwraw.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1}) #.limit(1000)


totalhw = 0 # total number of head words
for result in results:    
    word = result.get("hw")    
    if word:
        #print word
        fhw.write(word + '\n')
        totalhw += 1
    else:
        print "no head word"
        continue

fhw.close()
print "total head words: ", totalhw


# sort out by head words and output
# with homographs, 39195 head words have duplicates; unique number is 34398

# create mwl39195hw.txt
with codecs.open('E:\\1Now\\taglist\\mwl39195hwraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
    f.close()
lines = sorted(lines)
print 'hw unique number: ', len(set(lines))

fd = codecs.open('E:\\1Now\\taglist\\mwl39195hw.txt', 'w', 'utf-8')
cnt = 0
for line in lines:
    fd.write(line + '\n')
    cnt += 1
fd.close()
print cnt



# explore 39195 hws
with codecs.open('E:\\1Now\\taglist\\mwl39195hw.txt', 'r', 'utf-8') as f:
    words = f.read().splitlines()
    f.close()
print 'number of hws with homographs: ', len(words)
print 'number of unique hws: ', len(set(words))

# wordsC = [x.capitalize() for x in words]
wordsC = []
for word in words:
    # only capitalize words beginning with a lowercase letter. no need for 'Feburary'
    # Don't capitalize words like 'AWOL' -- Awol is not a word!
    if word[0] in string.lowercase[0:26]:
        wordsC.append(word.capitalize())
        cnt += 1
    
print 'number of words capitalized: ', len(wordsC)

# put words and wordsC together and get unique
# wordsTotal = sorted(set(words + wordsC))
# fd = codecs.open('E:\\1Now\\taglist\\mwl39195hwCap.txt', 'w', 'utf-8')
# for x in wordsTotal:
#     fd.write(x + '\n')
# fd.close()
# print len(wordsTotal)



total head words:  39195
hw unique number:  34398
39195
number of hws with homographs:  39195
number of unique hws:  34398
number of words capitalized:  37517
67125


In [6]:
# output mwl39195sum.txt  (having duplicates)
# hw entries and their summary (subhw, prn, idpvs, tags)
# 2015-11-27 2015-12-01

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest

fdict = codecs.open('E:\\1Now\\taglist\\mwl39195sumraw.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1, "subhw":1, "prn":1, "idpvs":1, "tag":1}) #.limit(1000)


totalhw = 0 # total number of head words
for result in results:    
    word = result.get("hw")    
    if word:
        #print word
        fdict.write(word)
        totalhw += 1
    else:
        print "no head word"
        continue

    # get items of result
    prns = result.get("prn")
    subhws = result.get("subhw")
    idpvs = result.get("idpvs")  
    tags = result.get('tag')
    
    if subhws:
        cnt = 1
        for subhw in subhws:
            if cnt == 1: fdict.write(' (' + subhw)                
            else: fdict.write(' ' + subhw)
            cnt += 1        
        fdict.write(')')
    
    if prns:
        cnt = 1
        for prn in prns:                 
            fdict.write(' /' + prn + '/')
            cnt += 1

    if idpvs:
        cnt = 1
        for idpv in idpvs:
            if cnt == 1: fdict.write('  [' + idpv)
            else: fdict.write(', ' + idpv)
            cnt += 1
        fdict.write(']')
            
    if tags:
        cnt = 1
        for tag in tags:
            if cnt == 1: fdict.write('  {' + tag)
            else: fdict.write(' ' + tag)
            cnt += 1
        fdict.write('}')

    fdict.write('\n')    

fhw.close()
fdict.close()
print "total head words: ", totalhw


# sort out and create mwl39195sum.txt
with codecs.open('E:\\1Now\\taglist\\mwl39195sumraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
    f.close()
lines = sorted(lines)
print 'hw summary unique number: ', len(set(lines))

fd = codecs.open('E:\\1Now\\taglist\\mwl39195sum.txt', 'w', 'utf-8')
cnt = 0
for line in lines:
    fd.write(line + '\n')
    cnt += 1
fd.close()
print cnt

total head words:  39195
hw unique number:  34398
39195
hw summary unique number:  38862
39195


In [78]:
# output mwlBenchmark.txt (mwBen) (no duplicates)

# format: derivatives of hw (inflections, or subhws);hw
# sort by 'derivatives;hw' and make every line unique
# 2015-11-28, 2015-11-30, 2015-12-01

# blazes;blaze
# blazing;blaze
# blazingly;blazing

# I can know blaze, blazing are hws, the two lines below are not allowd because hws have derivates:
# blaze;blaze
# blazing;blazing
# the two lines above are redundant.

# manually delete: asas

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest

fdict = codecs.open('E:\\1Now\\taglist\\mwlBenchmarkraw.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1, "infl":1, "subhw":1}) #.limit(1000)
# sort method will use much more memory
#results = geweiDict.find({}, {"_id":0, "hw":1, "prn":1, "alles":1, "idpvs":1, "sublist":1}).sort("hw", 1).limit(1000)


hwcnt = 0 # number of head words (hws)
# hwcapcnt = 0 # number of hws capitalized
inflcnt = 0 # number of inflections
subhwcnt = 0 # number of subhws
subhw1cnt = 0 # number of subhws whose first letter is not same as hw
onlyhwcnt = 0

for result in results:
    inflflag = 0
    subhwflag = 0
    
    word = result.get("hw")    
    if word:
        #print word
#         fdict.write(word + ';' + word + '\n')
        hwcnt += 1      

    else:
        print "no head word"
        continue

    # get items of result
    infls = result.get("infl")
    subhws = result.get("subhw")     
    
    if infls:
        for infl in infls:
            if infl == word:
#                 print 'infl == word', infl
                continue
            fdict.write(infl + ';' + word + '\n')
            inflcnt += 1   
            inflflag = 1
            
    if subhws:
        for subhw in subhws:
            # check if subhw is a real sub hw of word by comparing first letter 77561 -> 77256
            # no need again because the check has been done in db buidling
            #if subhw[0] != word[0]: continue       
            if subhw == word:
#                 print 'subhw == word', infl
                continue           
            if subhw[0] != word[0]:
#                 print 'subhw is not like word: ', subhw, '...', word
                subhw1cnt += 1
            fdict.write(subhw + ';' + word + '\n')
            subhwcnt += 1      
            subhwflag = 1
    if inflflag == 0 and subhwflag == 0:
        fdict.write(word + ';' + word + '\n')
        onlyhwcnt += 1
    
fdict.close()
print "total head words: ", hwcnt
# print 'total hws capitalized: ', hwcapcnt
print "total inflections: ", inflcnt
print 'total subhws: ', subhwcnt
print 'total subhws whose 1st letters are different from hws: ', subhw1cnt
print 'only hw: ', onlyhwcnt


# sort out by head words
with codecs.open('E:\\1Now\\taglist\\mwlBenchmarkraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
lines = sorted(set(lines))

fd = codecs.open('E:\\1Now\\taglist\\mwlBenchmark.txt', 'w', 'utf-8')
cnt = 0
for line in lines:
    fd.write(line + '\n')
    cnt += 1
fd.close()
print cnt

total head words:  39195
total inflections:  64355
total subhws:  9544
total subhws whose 1st letters are different from hws:  334
only hw:  11182
53972


In [10]:
# get duplicates
l = [1,2,3,4,4,5,5,6,1]
set([x for x in l if l.count(x) > 1])

{1, 4, 5}

In [74]:
# output core31.txt by processing core3000raw20151123.txt

with codecs.open('E:\\1Now\\taglist\\core3000raw20151123.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
    f.close()
words = [x.split('\t')[0] for x in lines]
wordsu = sorted(set(words))
fout = codecs.open('E:\\1Now\\taglist\\core3140n.txt', 'w', 'utf-8')
for word in wordsu:
    fout.write(word + '\n')
fout.close()

print len(words), len(wordsu)
dupl = set([x for x in words if words.count(x)>1])
print len(dupl)
# print sorted(dupl)

# # compare core3140n.txt and core31.txt
# with codecs.open('E:\\1Now\\taglist\\core31.txt', 'r', 'utf-8') as f:
#     core31s = f.read().splitlines()
#     f.close()
# print set(core31s) ^ set(wordsu)
    

3905 3140
661
set([])


In [1]:
def gethw():
    a = [{'k1': 'hello', 'k2':'hello2'}, {'k1': 'hi', 'k2':'hi2'}]
    return a
b = gethw()
for x in b:
    print x['k1'], '...', x['k2']

hello ... hello2
hi ... hi2


In [26]:
# check word lists only for building mwl mongodb database
# these word lists include (number in parentheses are the number of files)
#   * words of 18 topics from mwl website (18)
#   * 3140 core words from mwl website (1)
#   * words of "The Spelling of Different Sounds in English" (1)
#   * words of word roots, prefixes, suffixes (3)
#   sub-total: 23
#
#   * words of mwl single images (1)
#   * words of mwl group images (106)
#   sub-total: 107
#
#   * homographs (1)
#   * 3787 core words I collect (1)
#   sub-total: 2
# 
#   Total: 131



import codecs
import re
import string


def checkBymwBen(wordlist):    
    hwreturn = [] # hws returned by this function    
    
    # word list I want to check
    with codecs.open(wordlist, 'r', 'utf-8') as f:
        txtraw = f.read().splitlines()
        f.close()
    print '------', wordlist, '------'
    print '* has words : ', len(txtraw)
    print '* has unique word number: ', len(set(txtraw))
    print '* has duplicates: ', set([x for x in txtraw if txtraw.count(x) > 1])
    print '\n'

    # with codecs.open("E:\\1Now\\taglist\\mwl77Entry.txt", 'r', 'utf-8') as f:
    with codecs.open("E:\\1Now\\taglist\\mwlBenchmark.txt", 'r', 'utf-8') as f:    
        lines = f.read().splitlines()
        f.close()

    # create mwBen dictionary
    mwBens = []
    hws = [] # sorted unique hws
    hwsC = [] # sorted unique capitalized hws
    for line in lines:
        entry = {}
        hw = line.split(';')[1]
        entry['hw'] = hw
        entry['deriv'] = line.split(';')[0] # derivative, or hw that has no derivatives
        mwBens.append(entry)
        hws.append(hw)
        if hw[0] in string.lowercase[0:26]:
            hwsC.append(hw.capitalize())
        
    hws = sorted(set(hws))
    hwsC = sorted(set(hwsC))        

#     print 'mwBens: ', len(mwBens)
#     print 'hws: ', len(hws)
#     print 'hwsC: ', len(hwsC)


    # begin to check the word list against mw benchmark dictionary
    #  Word; Its-hw (Head Word)
    #
    # Three cases (the first two cases return hw)
    # 1. word is exactly same as hw or is a capitalized hw
    # 2. word is a derivative or is a capitalized derivative
    # 3. word is not in mwBen


    wordhw = [] # word is exactly same as hw
    wordhwC = [] # word is same as capitalized hw
    wordderiv = [] # derivatives or capitalized derivatives
    wordnotBen = [] # word is not in mwBen

    for word in txtraw:
        hwflag = 0
        derivflag = 0
        
        if word in hws:
            wordhw.append(word)
            hwreturn.append(word)
            hwflag = 1
        elif word in hwsC:
            wordhwC.append(word)
            hwreturn.append(word.lower())
            hwflag = 1

        if hwflag == 0:
            dercnt = 0
            for mwben in mwBens:                
                if word == mwben['deriv'] or word == mwben['deriv'].capitalize():
                    wordderiv.append(word)
                    hwreturn.append(mwben['hw'])
                    derivflag = 1
                    print 'derivative: ', word, '--- hw: ', mwben['hw']
                    dercnt += 1
            if dercnt > 1: print word, '--- derivate has multiple hws --- ', dercnt

        if hwflag == 0 and derivflag == 0:
            wordnotBen.append(word)
    
    print '\n'
    print '* same as hw: ', len(wordhw)  #,' -- ', wordhw
    print '* same as capitalized hw: ', len(wordhwC)  #,' -- ', wordhwC
    print '* has derivatives: ', len(wordderiv) #,' -- ', wordderiv
    print '* not in mwBen: ', len(wordnotBen), ' -- ', wordnotBen
    if len(txtraw) == len(wordhw) + len(wordhwC) + len(wordderiv) + len(wordnotBen): 
        print '--- len(txtraw) == len(wordhw) + len(wordhwC) + len(wordderiv) + len(wordnotBen) ---\n'
    else: print "--- words' numbers don't mattch, check ---\n"
    
    dupl = [x for x in hwreturn if hwreturn.count(x)>1]
    print 'how many hw duplicates--- ', len(dupl), ' - ', len(set(dupl)), ' = ', len(dupl)-len(set(dupl))
    print dupl

    print 'how many hws being returned--- ', len(txtraw) - len(wordnotBen) - (len(dupl)-len(set(dupl)))
    return sorted(set(hwreturn))
    
    
# result = checkBymwBen("E:\\1Now\\taglist\\core3140n.txt")
result = checkBymwBen('E:\\1Now\\taglist\\testmwBen.txt')

print len(result)
print result

------ E:\1Now\taglist\testmwBen.txt ------
* has words :  19
* has unique word number:  18
* has duplicates:  set([u'taking'])


derivative:  Taking --- hw:  take
derivative:  taking --- hw:  take
derivative:  taking --- hw:  take
derivative:  20/20 --- hw:  twenty-twenty
derivative:  Flurries --- hw:  flurry


* same as hw:  10
* same as capitalized hw:  3
* has derivatives:  5
* not in mwBen:  1  --  [u'tAke']
--- len(txtraw) == len(wordhw) + len(wordhwC) + len(wordderiv) + len(wordnotBen) ---

how many hw duplicates---  12  -  5  =  7
[u'take', u'take', u'takings', u'takings', u'take', u'blazing', u'blazing', u'take', u'twenty-twenty', u'twenty-twenty', u'impressionism', u'impressionism']
how many hws being returned---  11
11
[u'against', u'blaze', u'blazing', u'flurry', u'impressionism', u'revolt', u'take', u'taken', u'takings', u'took', u'twenty-twenty']
